**Summary**

Curve fitting can be approached in many different ways. This notebook compares my "standard" method---writing a function to return the sum of residual squares, and then minimizing it with `scipy.optimize.fmin`---with the python library `lmfit`, which in principle wraps around similar functions in the `scipy.optimize` library, and provides some extra functions, e.g. to constrain the optimized variables and to give confidence intervals. The example here comes from our recent attempts to use a model to predict protein adsorption to inline filters, based on an experiment.

The fitted parameters were notably quite different, although the curves in the end look roughly the same. This data set is probably not enough to fully compare the methods. The `lmfit` library provides some convenient functions and is fairly easy to implement, but unfortunately in this case setting parameter constraints caused the confidence interval function to fail.

**Import libraries and set plotting options:**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.stats import linregress
from scipy.optimize import fmin
import lmfit

plt.style.use('bmh')
%config InlineBackend.figure_format = 'svg'

**Raw data**

The data comes from an experiment monitoring the protein concentration after a single inline filter. The formulated protein has a nominal concentration of 5 mg/mL. The filter was wetted with the protein solution for a few minutes before the valve was opened to allow flow.

In [2]:
time = np.array([0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570, 600, 630, 660, 690, 720, 750, 780, 810, 840, 870, 900, 930, 960, 990, 1020, 1050, 1080, 1110, 1140, 1170, 1200, 1230, 1260, 1290, 1320, 1350, 1380, 1410, 1440, 1470, 1500])

volume = np.array([0, 13.2522, 22.8151, 35.4172, 48.2249, 63.3103, 73.558, 86.1327, 96.7225, 107.9742, 118.6576, 128.7267, 140.6426, 152.4861, 162.8935, 174.1857, 185.7268, 197.1192, 209.5242, 221.4678, 232.2868, 247.6275, 259.4228, 270.9724, 281.8857, 293.4212, 304.6134, 315.6425, 326.8809, 338.671, 348.127, 363.4559, 374.8323, 385.8517, 396.7044, 408.1897, 420.0901, 427.0895, 437.5906, 448.138, 459.2603, 469.2068, 480.2171, 491.9888, 502.4742, 513.0803, 524.2078, 535.2854, 545.8359, 555.0863, 564.3464])

concentration = np.array([3.023798609, 3.114830065, 3.193132997, 3.344098032, 3.545952177, 3.71131258, 3.880179942, 4.051567078, 4.145738602, 4.204649925, 4.310641861, 4.373957515, 4.449874592, 4.523475265, 4.557658434, 4.621247387, 4.665646744, 4.688958049, 4.724696445, 4.763434982, 4.777761936, 4.817210579, 4.840685558, 4.846937061, 4.861563492, 4.862069321, 4.865633011, 4.883685875, 4.888158989, 4.893807769, 4.89773283, 4.902150154, 4.908982992, 4.91435442, 4.919867516, 4.916160679, 4.928572059, 4.917313385, 4.922553825, 4.919561267, 4.92966497, 4.934463024, 4.935445404, 4.937192059, 4.930878401, 4.932299042, 4.933749437, 4.929975319, 4.935555362, 4.937137485, 4.940053844])

**Additional constants**

$c$ denotes the concentration in the liquid phase, $q$ the concentration in the adsorbed phase.

Note that $q_{initial}$ comes from a mass balance from the initial equilibration period (no raw data).

In [3]:
Vol = 670 # mL
Vdot = linregress(time, volume).slope # mL/s
c_input = 4.94 # mg/mL
c_init = concentration[0] # ca. 3 mg/mL
q_init = c_input - c_init

**Part 1: Using `lmfit` to fit the single-filter data**

Concentration in liquid phase, $c$
$$\frac{dc}{dt} = \frac{\dot{V}}{V} (c_{input} - c) - k_1 c (q_{max} - q) + k_2 q$$

Concentration in adsorbed phase, $q$
$$\frac{dq}{dt} = k_1 c (q_{max} - q) - k_2 q$$

Note: at equilibrium $\frac{dq}{dt} = 0$ and $q = q_{max} \frac{(k_1/k_2) c_e}{1 + (k_1/k_2) c_e}$ (Langmuir isotherm)

Hashim, M. A.; Chu, K. H. Prediction of Protein Breakthrough Behavior Using Simplified Analytical Solutions. *Sep. Purif. Technol.* **2007**, 53 (2), 189–197 DOI: 10.1016/j.seppur.2006.06.028.

In [4]:
def rates(state, t, paras):
    c, q = state

    k1 = paras[0]
    k2 = paras[1]
    qmax = paras[2]
    V = paras[3]

    dcdt = Vdot / V * (c_input - c) - k1 * c * (qmax - q) + k2 * q
    dqdt = k1 * c * (qmax - q) - k2 * q
    return [dcdt, dqdt]


def conc(t, c0, q0, k1, k2, qmax, V):
    initial_state = (c0, q0)
    params_list = [k1, k2, qmax, V]
    x = odeint(rates, initial_state, t, args=(params_list, ))
    return x

**Additional functions**

I had to write this function, which will be used to create the `lmfit.Model` (a class), because the previously defined `conc` function returns both $c$ and $q$, while we only have data for $c$. When I tried to use `conc` directly for the Model, an error gets thrown during fitting because of the different dimensions of the output array and the data array. There is probably a cleaner way to address this.

In [5]:
def conc_index(t, c0, q0, k1, k2, qmax, index, V=Vol):
    x = conc(t, c0, q0, k1, k2, qmax, V)
    return x[:,index]

**Using the `lmfit.Model` class**

We make the object `c_model` by calling the `lmfit.Model` class, with the `conc_index` function passed as the model function, and the independent variable explicitly identified.

In the line that is commented out, I originally used the index as an independent variable (the index variable is only used to pick out the $c$ or $q$ values). This makes more logical sense; the only problem is that when you have more than one independent variable, you lose some of the convenient plotting methods of the `Model` class. So I faked it this way, I just have to be careful to make sure that the fit function does not try to change the index variable.

In [6]:
c_model = lmfit.Model(conc_index, independent_vars='t')

# c_model = lmfit.Model(conc_index, independent_vars=['t','index'])

**Setting Parameters**

The `make_params()` method will take the rest of the variables that were not marked as independent when making the `Model` object and turn them into `Parameter` objects, stored in the `params` object. Once they are made, we can set their initial values with the `value` attribute, and whether they vary or not. We can also set `min` and `max` attributes later.

For now we will fix $index$, $c_0$, $q_0$, and $V$, and allow $k_1$, $k_2$, and $q_{max}$ to vary.

In [7]:
params = c_model.make_params()

params['index'].value = 0 # This will return the c values
params['index'].vary = False

params['k1'].value = 0.01

params['k2'].value = 0.01

params['qmax'].value = 1

params['c0'].value = c_init
params['c0'].vary = False

params['q0'].value = q_init
params['q0'].vary = False

params['V'].vary = False

**Fitting the model**

Now we can use the `fit()` method and print out the fit report. The first function parameter is the data, the second is the params object defined above, and the third is the independent variable.

In [8]:
lmfit_results = c_model.fit(concentration,params,t=time)
print(lmfit_results.fit_report())

[[Model]]
    Model(conc_index)
[[Fit Statistics]]
    # function evals   = 81
    # data points      = 51
    # variables        = 3
    chi-square         = 0.039
    reduced chi-square = 0.001
    Akaike info crit   = -359.414
    Bayesian info crit = -353.618
[[Variables]]
    c0:      3.023799 (fixed)
    q0:      1.916201 (fixed)
    k1:      0.00222362 +/- 0.000193 (8.67%) (init= 0.01)
    k2:     -0.00437038 +/- 0.000633 (14.49%) (init= 0.01)
    qmax:    0.18167960 +/- 0.006684 (3.68%) (init= 1)
    index:   0 (fixed)
    V:       670 (fixed)
[[Correlations]] (unreported correlations are <  0.100)
    C(k1, k2)                    = -0.997 
    C(k2, qmax)                  =  0.739 
    C(k1, qmax)                  = -0.705 



**Visual check of goodness of fit**

The `lmfit` function has a convenient `plot()` function that shows the residuals and the fitted line along with the curve generated using the initial values.

In [10]:
lmfit_results.plot()

(<matplotlib.figure.Figure at 0xbaf2b70>,
 <matplotlib.gridspec.GridSpec at 0xbae7eb8>)

Worth noting is that the slight S-shape of the data is not reproduced in the fit. In general I think that the breakthrough curves that typically result from this type of model do show S-shapes, but in this case not with our values.

**Confidence intervals**

The `ci_report()` method runs the confidence interval analysis and gives a report. There is a more advanced implementation on the documentation site (https://lmfit.github.io/lmfit-py/confidence.html) that I would like to try at some point.

In [11]:
print(lmfit_results.ci_report())

C:\Users\pyao1\AppData\Local\Continuum\miniconda3\lib\site-packages\scipy\integrate\odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


         99.73%    95.45%    68.27%    _BEST_    68.27%    95.45%    99.73%
 k1  :  -0.00061  -0.00040  -0.00020   0.00222  +0.00020  +0.00042  +0.00065
 k2  :  -0.00210  -0.00135  -0.00066  -0.00437  +0.00065  +0.00132  +0.00202
 qmax:  -0.01813  -0.01231  -0.00636   0.18168  +0.00717  +0.01575  +0.02676


**Constrained optimization**

Here I will set the minimum values for $k_1$, $k_2$, and $q_{max}$ as 0. Note that in the previous unconstrained fit, the resulting $k_2$ was negative. From earlier analysis we know already that we can eliminate this parameter completely (this changes the so-called *Thomas model* to the *Bohart-Adams model* with a rectangular instead of Langmuir isotherm, see the earlier reference).

Unfortunately in this case the confidence intervals cannot be calculated. I am not sure of the precise reason, whether this happens any time the variables are constrained, or whether these fitted values are too close to the constraints, etc.

In [12]:
params['k1'].min = 0
params['k2'].min = 0
params['qmax'].min = 0

bound_results = c_model.fit(concentration,params,t=time)
print(bound_results.fit_report())

[[Model]]
    Model(conc_index)
[[Fit Statistics]]
    # function evals   = 54
    # data points      = 51
    # variables        = 3
    chi-square         = 0.079
    reduced chi-square = 0.002
    Akaike info crit   = -324.021
    Bayesian info crit = -318.225
[[Variables]]
    c0:      3.023799 (fixed)
    q0:      1.916201 (fixed)
    k1:      0.00092191 +/- 0        (0.00%) (init= 0.01)
    k2:      4.2343e-11 +/- 0        (0.00%) (init= 0.01)
    qmax:    0.27321401 +/- 0        (0.00%) (init= 1)
    index:   0 (fixed)
    V:       670 (fixed)



The commented-out function call `print(bound_results.ci_report())` will give the error below if it's run.

In [13]:
# print(bound_results.ci_report())

# MinimizerException: 
# Cannot determine Confidence Intervals without sensible uncertainty estimates

We will store the values from the fitting results for later use.

In [14]:
k1_lmfit = lmfit_results.params['k1'].value
k2_lmfit = lmfit_results.params['k2'].value
qmax_lmfit = lmfit_results.params['qmax'].value

k1_bound = bound_results.params['k1'].value
k2_bound = bound_results.params['k2'].value
qmax_bound = bound_results.params['qmax'].value

**Part 2: Using `fmin` to fit the single-filter data, and fit comparison**

I define the `residual_sq` function to return the sum of squared residuals. I suspect this may not be the "textbook" way to do it; I took this code from when I was doing cake-filtration calculations.

In [15]:
def residual_sq(paras, t, c0, q0, V, data):
    k1 = paras[0]
    k2 = paras[1]
    qmax = paras[2]
    
    model = conc(t, c0, q0, k1, k2, qmax, V)
    model_c = model[:, 0]
    e = ((model_c - data) ** 2).sum()
    return e

**Fit with `fmin`**

The fitted values are stored directly into the variables that I will use later. To use `fmin` I pass  the `residual_sq` function to be minimized, the initial guess values, and then the rest of the arguments that the `residual_sq` function needs (including the data).

In [16]:
k1_fmin, k2_fmin, qmax_fmin = fmin(residual_sq, [0.01, 0.01, 1], args=(time, c_init, q_init, Vol, concentration))

Optimization terminated successfully.
         Current function value: 0.151888
         Iterations: 77
         Function evaluations: 145


**Comparison of fitted values**

If we take a quick look at the values, we see they are quite different:

In [17]:
print([k1_lmfit, k2_lmfit, qmax_lmfit],'\n',[k1_bound, k2_bound, qmax_bound],'\n',[k1_fmin, k2_fmin, qmax_fmin])

[0.002223625849451237, -0.004370385963982178, 0.18167960541588402] 
 [0.0009219078152304938, 4.234346206999362e-11, 0.2732140108792074] 
 [7.603112915851363e-05, 0.002924617255912217, 2.149016889498464]


Let's also compare the sum of residual squares using the function previously defined:

In [18]:
rsq_lmfit = residual_sq([k1_lmfit, k2_lmfit, qmax_lmfit], time, c_init, q_init, Vol, concentration)
rsq_bound = residual_sq([k1_bound, k2_bound, qmax_bound], time, c_init, q_init, Vol, concentration)
rsq_fmin = residual_sq([k1_fmin, k2_fmin, qmax_fmin], time, c_init, q_init, Vol, concentration)

print([rsq_lmfit, rsq_bound, rsq_fmin])

[0.03943304129569024, 0.07893182211672405, 0.15188792723317143]


So the unconstrained `lmfit` results have the lowest sum, followed by the constrained `lmfit`, and finally `fmin`.

**Evaluating the model with the fitted parameters**

The model function can be evaluated several ways.

The `ModelResults` class has an `eval()` method that automatically uses the fitted values:

In [19]:
lmfit_results.eval(t=time)

array([3.02379861, 3.15760417, 3.2954761 , 3.43524282, 3.57458177,
       3.71117589, 3.84287054, 3.9678086 , 4.08452618, 4.19200116,
       4.28965464, 4.37731361, 4.45514775, 4.52359292, 4.58327315,
       4.63492904, 4.67935752, 4.71736482, 4.74973225, 4.77719335,
       4.80042034, 4.82001775, 4.83652115, 4.8503993 , 4.86205851,
       4.87184817, 4.88006681, 4.88696813, 4.8927668 , 4.89764381,
       4.90175129, 4.90521685, 4.90814724, 4.91063168, 4.9127446 ,
       4.91454802, 4.91609361, 4.91742434, 4.91857595, 4.91957813,
       4.92045558, 4.92122877, 4.92191474, 4.92252762, 4.92307914,
       4.92357905, 4.92403544, 4.92445502, 4.92484339, 4.92520517,
       4.92554423])

We can also use the `eval` method of the `Model` class and pass the fitted parameters. It gives the same values:

In [20]:
c_model.eval(lmfit_results.params,t=time) == lmfit_results.eval(t=time)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

We can also manually pass in parameter values. This is what I will use because I am going to use fitted parameters from the `fmin` method, which does not use the `Parameter` class.

In [21]:
c_model.eval(t=time, c0=c_init, q0=q_init, k1=k1_lmfit, k2=k2_lmfit, qmax=qmax_lmfit, index=0, V=Vol) == lmfit_results.eval(t=time)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

Just to check that we get the same results when we call the original `conc` function:

In [22]:
conc(time, c_init, q_init, k1_lmfit, k2_lmfit, qmax_lmfit, Vol)[:,0] == lmfit_results.eval(t=time)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

Generate the $c$ and $q$ values from the three fitting runs:

In [23]:
c_lmfit = c_model.eval(t=time, c0=c_init, q0=q_init, k1=k1_lmfit, k2=k2_lmfit, qmax=qmax_lmfit, index=0, V=Vol)
q_lmfit = c_model.eval(t=time, c0=c_init, q0=q_init, k1=k1_lmfit, k2=k2_lmfit, qmax=qmax_lmfit, index=1, V=Vol)

c_bound = c_model.eval(t=time, c0=c_init, q0=q_init, k1=k1_bound, k2=k2_bound, qmax=qmax_bound, index=0, V=Vol)
q_bound = c_model.eval(t=time, c0=c_init, q0=q_init, k1=k1_bound, k2=k2_bound, qmax=qmax_bound, index=1, V=Vol)

c_fmin = c_model.eval(t=time, c0=c_init, q0=q_init, k1=k1_fmin, k2=k2_fmin, qmax=qmax_fmin, index=0, V=Vol)
q_fmin = c_model.eval(t=time, c0=c_init, q0=q_init, k1=k1_fmin, k2=k2_fmin, qmax=qmax_fmin, index=1, V=Vol)

Then let's plot them next to each other:

In [24]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3)

ax1.plot(time,c_lmfit)
ax1.plot(time, concentration, marker='o', markerfacecolor='None', linestyle='None', markersize=4)
ax1.set_title('lmfit')
ax1.set_ylabel('c (mg/mL)')

ax2.plot(time,c_bound)
ax2.plot(time, concentration, marker='o', markerfacecolor='None', linestyle='None', markersize=4)
ax2.set_title('bound')

ax3.plot(time,c_fmin)
ax3.plot(time, concentration, marker='o', markerfacecolor='None', linestyle='None', markersize=4)
ax3.set_title('fmin')

ax4.plot(time,q_lmfit)
ax4.set_ylabel('q (mg/mL)')

ax5.plot(time,q_bound)
ax5.set_xlabel('Time (s)')

ax6.plot(time,q_fmin)


In all cases we have this "unphysical" result that the adsorbed concentration $q$ *decreases* from the initial value. Back when I was guessing different values for $V$ and $q_{initial}$ I had nicer looking plots with $V = 50$ and $q_{initial} = 30$, but I had less physical basis for those values.

**Part 3: Simulating two filters in series**

The rates are defined in a similar way to the single filter. For the input of filter 2 we use the output of filter 1. Naturally we assume that $k_1$, $k_2$, and $q_{max}$ are the same for both filters.

As a side note, I don't need to do some of the stupid stuff I did in the earlier function definitions, with several (positional) arguments to the `conc` function that get packed up in a list to pass to the `rates` function and then unpacked again. I did it that way earlier because I'm still not completely sure about the syntax of the `lmfit` functions, especially how it identifies the model parameters.

In [25]:
def rates_twofilters(state, t, paras):
    c1, q1, c2, q2 = state

    k1 = paras[0]
    k2 = paras[1]
    qmax = paras[2]
    V = paras[3]

    dc1dt = Vdot / V * (c_input - c1) - k1 * c1 * (qmax - q1) + k2 * q1
    dq1dt = k1 * c1 * (qmax - q1) - k2 * q1

    dc2dt = Vdot / V * (c1 - c2) - k1 * c2 * (qmax - q2) + k2 * q2
    dq2dt = k1 * c2 * (qmax - q2) - k2 * q2

    return [dc1dt, dq1dt, dc2dt, dq2dt]


def conc_twofilters(t, initial_state, paras):
    x = odeint(rates_twofilters, initial_state, t, args=(paras,))
    return x

For now we will assume that $c_{1,initial} = c_{2,initial}$ and that filter 2 has no adsorbed material to start. We'll simulate a bit longer than the experiment and plot against the volume of filtrate (`v_sim`).

In [26]:
init_state_twofilters = c_init, q_init, c_init, 0

t_sim = np.arange(0, 6000, 5)
v_sim = t_sim * Vdot

For now I just evaluate with the `conc_twofilters` function but it would probably be good to use the `Model` class again, even though I'm not fitting any data here. I'll do it later.

In [27]:
twofilters_lmfit = conc_twofilters(t_sim, init_state_twofilters, [k1_lmfit, k2_lmfit, qmax_lmfit, Vol])
twofilters_bound = conc_twofilters(t_sim, init_state_twofilters, [k1_bound, k2_bound, qmax_bound, Vol])
twofilters_fmin = conc_twofilters(t_sim, init_state_twofilters, [k1_fmin, k2_fmin, qmax_fmin, Vol])

Now let's plot the results:

In [30]:
fig2, ((ax7, ax8, ax9), (ax10, ax11, ax12)) = plt.subplots(2, 3)

ax7.plot(v_sim,twofilters_lmfit[:,2])
ax7.hlines(4.7, v_sim[0], v_sim[-1], linestyle='dotted')
ax7.set_title('lmfit')
ax7.set_ylabel('c2 (mg/mL)')

ax8.plot(v_sim,twofilters_bound[:,2])
ax8.hlines(4.7, v_sim[0], v_sim[-1], linestyle='dotted')
ax8.set_title('bound')

ax9.plot(v_sim,twofilters_fmin[:,2])
ax9.hlines(4.7, v_sim[0], v_sim[-1], linestyle='dotted')
ax9.set_title('fmin')

ax10.plot(v_sim,twofilters_lmfit[:,3])
ax10.set_ylabel('q2 (mg/mL)')

ax11.plot(v_sim,twofilters_bound[:,3])
ax11.set_xlabel('Volume (mL)')

ax12.plot(v_sim,twofilters_fmin[:,3])

So the $c$ plots only differ substantially in the initial "dip" in concentration. In reality I suspect we will not be able to observe this, because this model does not account for the significant amount of time needed to fill the filter before it comes to the outlet (filter volume 670 mL with a flow rate of 0.38 mL/s).

There is a more pronounced difference in the adsorbed phase concentration $q$ but we cannot measure this directly.

**Notes about `lmfit`**

There are several fitting methods that can be used. As noted in the documentation (copied below), most of them come from the `scipy.optimize.minimize()` function.

In [31]:
print(c_model.fit(concentration,params,t=time,method='nelder').fit_report())

[[Model]]
    Model(conc_index)
[[Fit Statistics]]
    # function evals   = 469
    # data points      = 51
    # variables        = 3
    chi-square         = 0.079
    reduced chi-square = 0.002
    Akaike info crit   = -324.021
    Bayesian info crit = -318.225
[[Variables]]
    c0:      3.023799 (fixed)
    q0:      1.916201 (fixed)
    k1:      0.00092187 (init= 0.01)
    k2:      6.0819e-10 (init= 0.01)
    qmax:    0.27321371 (init= 1)
    index:   0 (fixed)
    V:       670 (fixed)



https://lmfit.github.io/lmfit-py/fitting.html

method (str, optional) –  
Name of the fitting method to use. Valid values are:

‘leastsq’: Levenberg-Marquardt (default)  
‘least_squares’: Least-Squares minimization, using Trust Region Reflective method by default  
‘differential_evolution’: differential evolution  
‘brute’: brute force method  
‘nelder‘: Nelder-Mead  
‘lbfgsb’: L-BFGS-B  
‘powell’: Powell  
‘cg’: Conjugate-Gradient  
‘newton’: Newton-Congugate-Gradient  
‘cobyla’: Cobyla  
‘tnc’: Truncate Newton  
‘trust-ncg’: Trust Newton-Congugate-Gradient  
‘dogleg’: Dogleg  
‘slsqp’: Sequential Linear Squares Programming  
In most cases, these methods wrap and use the method of the same name from scipy.optimize, or use scipy.optimize.minimize with the same method argument. Thus ‘leastsq‘ will use scipy.optimize.leastsq, while ‘powell‘ will use scipy.optimize.minimizer(...., method=’powell’)